In [6]:
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 3.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.6 MB 3.7 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.6 MB 4.0 MB/s eta 0:00:03
   ------------ --------------------------- 3.7/11.6 MB 4.0 MB/s eta 0:00:02
   --------------- ------------------------ 4.5/11.6 MB 4.1 MB/s eta 0:00:02
   ------------------ --------------------- 5.5/11.6 MB 4.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.6 MB 4.4 MB/s eta 0:00:02
   -------------------------- ------------- 7.6/11.6 MB 4.6 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.6 MB 4.7 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.6 MB 4.7 MB/s eta 0:00:01
   --------------------

In [10]:
from sklearn import *
import pandas as pd

df = pd.read_csv("../treedata.csv")
df = df.drop(columns=["ObjectID", "Other.1"], axis=1)

# One-hot encode categorical columns
categorical_cols = ["Tree Species", "Condition", "Site Factors", "Type of Soil", "Weather factors", "Failed Part", "Root Failure", "Stem Failure", "Branch Failure", "Location and Percentage of Decay", "Decay Present"]
df = pd.get_dummies(df, columns=categorical_cols)

# Convert one-hot encoded columns to integers
one_hot_cols = [col for col in df.columns if col.startswith(tuple(categorical_cols))]
df[one_hot_cols] = df[one_hot_cols].applymap(lambda x: int(x))

print(df.head())

   Diameter of Tree  Height of Tree  Diameter of Failed Part  \
0                27              90                        5   
1                34              40                       12   
2                30             135                       30   
3                30             110                       30   
4                14              25                       14   

   Length of Failed Part  Tree Species_AbiesGrandis  \
0                   20.0                          0   
1                   40.0                          0   
2                  135.0                          0   
3                  110.0                          0   
4                   25.0                          0   

   Tree Species_AcerCircinatum  Tree Species_AcerMacrophyllum  \
0                            0                              0   
1                            0                              1   
2                            0                              0   
3                       

C:\Users\eoinm\AppData\Local\Temp\ipykernel_22988\3995165576.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[one_hot_cols] = df[one_hot_cols].applymap(lambda x: int(x))
